<a href="https://colab.research.google.com/github/plus2net/Python-basics/blob/main/Mutual_Fund_NAV_Tracking_1_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://www.plus2net.com/images/top2.jpg)        More on  [Mutual fund NAV tracking ](https://www.plus2net.com/python/bs4-automate-mutual-fund-nav-tracking-mfapi.php)

Get the current NAV of a fund by entering the scheme code.

In [2]:
import requests

def get_latest_nav(scheme_code):
    url = f"https://api.mfapi.in/mf/{scheme_code}/latest"
    response = requests.get(url)
    data = response.json()

    latest_nav = data['data'][0]['nav']
    date = data['data'][0]['date']
    fund_name = data['meta']['scheme_name']

    return fund_name, latest_nav, date

# Example: Quant Small Cap Fund (Direct Plan)
name, nav, date = get_latest_nav("120847")
print(f"Fund: {name} | Current NAV: {nav} | Date: {date}")

Fund: quant ELSS Tax Saver Fund - Growth Option - Direct Plan | Current NAV: 409.90870 | Date: 16-01-2026
